In [1]:
from pycocotools.coco import COCO
from pathlib import Path

In [2]:
coco_ann_dir = Path("/home/p0w3r/gitrepos/TERAN_fork/data/coco/annotations/")
assert coco_ann_dir.exists()

In [3]:
train_caps_json = coco_ann_dir.joinpath('captions_train2014.json')
val_caps_json = coco_ann_dir.joinpath('captions_val2014.json')
assert train_caps_json.exists()
assert val_caps_json.exists()

In [4]:
train_coco = COCO(train_caps_json)

loading annotations into memory...
Done (t=0.58s)
creating index...
index created!


In [5]:
val_coco = COCO(val_caps_json)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [6]:
len(val_coco.anns.keys())

202654

In [7]:
len(train_coco.anns.keys())

414113

In [8]:
val_caps = [val_coco.anns[ann_id]['caption'] for ann_id in val_coco.anns.keys()]

In [9]:
print(val_caps[1])

A black Honda motorcycle parked in front of a garage.


In [10]:
val_coco.imgToAnns[328838]

[{'image_id': 328838,
  'id': 762586,
  'caption': 'a short stack of pancakes sitting on a black plate '},
 {'image_id': 328838,
  'id': 764020,
  'caption': 'A black plate with pancakes and syrup on it.'},
 {'image_id': 328838,
  'id': 765907,
  'caption': 'A stack of pan cakes sitting on top of a black table.'},
 {'image_id': 328838,
  'id': 768103,
  'caption': 'pancakes sitting on a pizza stone, covered in butter. '},
 {'image_id': 328838,
  'id': 769801,
  'caption': 'A stack of four pancakes on a skillet.'}]

# create a dataframe from the JSON files
##### cols: image_id, caption_id, caption, origin (train_2014 or val_2014)

In [11]:
import pandas as pd

In [12]:
list(val_coco.imgToAnns.values())[0]

[{'image_id': 203564,
  'id': 37,
  'caption': 'A bicycle replica with a clock as the front wheel.'},
 {'image_id': 203564, 'id': 181, 'caption': 'The bike has a clock as a tire.'},
 {'image_id': 203564,
  'id': 478,
  'caption': 'A black metal bicycle with a clock inside the front wheel.'},
 {'image_id': 203564,
  'id': 6637,
  'caption': 'A bicycle figurine in which the front wheel is replaced with a clock\n'},
 {'image_id': 203564,
  'id': 6802,
  'caption': 'A clock with the appearance of the wheel of a bicycle '}]

In [13]:
val_data = []
for val in val_coco.imgToAnns.values():
    for v in val:
        val_data.append(v)

In [14]:
val_df = pd.DataFrame(val_data)

In [15]:
val_df.rename(columns = {'id': 'caption_id'}, inplace=True)
val_df['origin'] = ['val_2014'] * len(val_df)
val_df.head()

,image_id,caption_id,caption,origin
0,203564,37,A bicycle replica with a clock as the front wh...,val_2014
1,203564,181,The bike has a clock as a tire.,val_2014
2,203564,478,A black metal bicycle with a clock inside the ...,val_2014
3,203564,6637,A bicycle figurine in which the front wheel is...,val_2014
4,203564,6802,A clock with the appearance of the wheel of a ...,val_2014


In [16]:
train_data = []
for train in train_coco.imgToAnns.values():
    for t in train:
        train_data.append(t)

len(train_data)

414113

In [17]:
train_df = pd.DataFrame(train_data)
train_df.rename(columns = {'id': 'caption_id'}, inplace=True)
train_df['origin'] = ['train_2014'] * len(train_df)
train_df.head()

,image_id,caption_id,caption,origin
0,318556,48,A very clean and well decorated empty bathroom,train_2014
1,318556,126,A blue and white bathroom with butterfly theme...,train_2014
2,318556,219,A bathroom with a border of butterflies and bl...,train_2014
3,318556,255,An angled view of a beautifully decorated bath...,train_2014
4,318556,3555,A clock that blends in with the wall hangs in ...,train_2014


# Persist the Dataframe

In [19]:
coco_df = train_df.copy().append(val_df.copy())
coco_df.reset_index(drop=True).to_feather('../data/coco_raw.df.feather')

In [20]:
len(coco_df) // 5

123353

##### Why are there only 123353 images in the dataset when the website and paper says 328k?!

In [21]:
coco_df = pd.read_feather('../data/coco_raw.df.feather')

# generate metadata for the captions WITH SPACY

In [21]:
import sys
sys.path.append('..')
from utils import generate_caption_stats

pos_tags_stats = True
readability_scores = False
n_spacy_workers = 8

coco_metadata_spacy = generate_caption_stats(coco_df, pos_tags_stats, readability_scores, n_spacy_workers, backend='spacy')

print(coco_metadata_spacy.head())
coco_metadata_spacy.reset_index(drop=True).to_feather('../data/coco_metadata_spacy.df.feather')

2021-01-20 19:30:12.831 | INFO     | utils:generate_caption_stats:135 - Generating caption statistics...
100%|██████████| 616767/616767 [04:26<00:00, 2317.34it/s]
2021-01-20 19:34:45.959 | INFO     | utils:generate_caption_stats:380 - Finished adding caption statistics in 273.1276879310608 seconds!


   image_id  caption_id                                            caption  \
0    318556          48     A very clean and well decorated empty bathroom   
1    318556         126  A blue and white bathroom with butterfly theme...   
2    318556         219  A bathroom with a border of butterflies and bl...   
3    318556         255  An angled view of a beautifully decorated bath...   
4    318556        3555  A clock that blends in with the wall hangs in ...   

       origin  num_tok  num_sent  min_sent_len  max_sent_len  num_ne ne_types  \
0  train_2014        8         1             8             8       0       []   
1  train_2014       11         1            11            11       0       []   
2  train_2014       16         1            16            16       0       []   
3  train_2014        9         1             9             9       0       []   
4  train_2014       13         1            13            13       0       []   

   ... num_conj  num_verb  num_sym  num_num 

# generate metadata for the captions WITH NLTK

In [22]:
pos_tags_stats = True
readability_scores = False
n_spacy_workers = 8

coco_metadata_nltk = generate_caption_stats(coco_df, pos_tags_stats, readability_scores, n_spacy_workers, backend='nltk')

print(coco_metadata_nltk.head())
coco_metadata_nltk.reset_index(drop=True).to_feather('../data/coco_metadata_nltk.df.feather')

2021-01-20 19:34:46.953 | INFO     | utils:generate_caption_stats:135 - Generating caption statistics...
100%|██████████| 616767/616767 [34:25<00:00, 298.60it/s]
2021-01-20 20:09:15.748 | INFO     | utils:generate_caption_stats:380 - Finished adding caption statistics in 2068.794457912445 seconds!


   image_id  caption_id                                            caption  \
0    318556          48     A very clean and well decorated empty bathroom   
1    318556         126  A blue and white bathroom with butterfly theme...   
2    318556         219  A bathroom with a border of butterflies and bl...   
3    318556         255  An angled view of a beautifully decorated bath...   
4    318556        3555  A clock that blends in with the wall hangs in ...   

       origin  num_tok  num_sent  min_sent_len  max_sent_len  num_ne ne_types  \
0  train_2014        8         1             8             8       0       []   
1  train_2014       11         1            11            11       0       []   
2  train_2014       16         1            16            16       0       []   
3  train_2014        9         1             9             9       0       []   
4  train_2014       13         1            13            13       0       []   

   ... num_conj  num_verb  num_sym  num_num 

# generate metadata for the captions WITH POLYGLOT

In [23]:
import sys
sys.path.append('..')
from utils import generate_caption_stats

pos_tags_stats = True
readability_scores = False
n_spacy_workers = 8

coco_metadata_polyglot = generate_caption_stats(coco_df, pos_tags_stats, readability_scores, n_spacy_workers, backend='polyglot')

print(coco_metadata_polyglot.head())
coco_metadata_polyglot.reset_index(drop=True).to_feather('../data/coco_metadata_polyglot.df.feather')

2021-01-25 10:41:36.031 | INFO     | utils:generate_caption_stats:146 - Generating caption statistics using POLYGLOT...
  0%|          | 0/616767 [00:00<?, ?it/s]

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /home/p0w3r/polyglot_data...
[polyglot_data]   Package embeddings2.en is already up-to-date!
[polyglot_data] Downloading package ner2.en to
[polyglot_data]     /home/p0w3r/polyglot_data...
[polyglot_data]   Package ner2.en is already up-to-date!
[polyglot_data] Downloading package pos2.en to
[polyglot_data]     /home/p0w3r/polyglot_data...
[polyglot_data]   Package pos2.en is already up-to-date!


 13%|█▎        | 83045/616767 [07:08<39:23, 225.85it/s]Detector is not able to detect the language reliably.


 18%|█▊        | 112526/616767 [09:45<44:57, 186.92it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 21%|██        | 129570/616767 [11:12<56:51, 142.82it/s]  Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 24%|██▍       | 149332/616767 [13:22<40:26, 192.64it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 24%|██▍       | 149859/616767 [13:25<1:19:52, 97.43it/s] Detector is not able to detect the language reliably.


 31%|███       | 191421/616767 [18:03<42:05, 168.45it/s]  Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 34%|███▍      | 211973/616767 [20:22<1:01:09, 110.32it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 34%|███▍      | 212309/616767 [20:24<39:16, 171.65it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 35%|███▌      | 216083/616767 [20:57<1:18:58, 84.55it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 36%|███▌      | 221124/616767 [21:32<31:20, 210.45it/s]  Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 36%|███▌      | 222048/616767 [21:37<36:35, 179.82it/s]Detector is not able to detect the language reliably.
Detector 

...

 82%|████████▏ | 505784/616767 [54:11<12:58, 142.55it/s]Detector is not able to detect the language reliably.


 89%|████████▉ | 547523/616767 [58:35<07:13, 159.70it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 90%|████████▉ | 555071/616767 [59:32<14:17, 71.98it/s] Detector is not able to detect the language reliably.


 91%|█████████ | 560743/616767 [1:00:13<04:48, 194.31it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 95%|█████████▌| 588017/616767 [1:03:01<02:59, 160.43it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 98%|█████████▊| 606085/616767 [1:04:58<00:56, 188.71it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 99%|█████████▉| 609989/616767 [1:05:22<00:56, 120.91it/s]Detector is not able to detect the language reliably.


100%|██████████| 616767/616767 [1:07:01<00:00, 153.36it/s]


   image_id  caption_id                                            caption  \
0    318556          48     A very clean and well decorated empty bathroom   
1    318556         126  A blue and white bathroom with butterfly theme...   
2    318556         219  A bathroom with a border of butterflies and bl...   
3    318556         255  An angled view of a beautifully decorated bath...   
4    318556        3555  A clock that blends in with the wall hangs in ...   

       origin  num_tok  num_sent  min_sent_len  max_sent_len  num_ne ne_types  \
0  train_2014        8         1             8             8       0       []   
1  train_2014       11         1            11            11       0       []   
2  train_2014       16         1            16            16       0       []   
3  train_2014        9         1             9             9       0       []   
4  train_2014       13         1            13            13       0       []   

   ... num_conj  num_verb  num_sym  num_num 